**Data preprocessing, Create Label and Features**

In [1]:
#import dependencies
import cv2
import numpy as np
import os
from matplotlib import pyplot as pyplot
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
DATA_PATH = os.path.join("MP_DATA") # paht of the exported data
with open("word_list.txt") as file:
  action = file.readlines()
for i,word in enumerate(action):
  action[i] = word.replace("\n","")
actions = np.array(action)

no_sequences = 30 # take 30 video to collect data for each action
sequence_length = 30

In [10]:
sequences,lables = [], []
actions = np.array(os.listdir(os.path.join(DATA_PATH)))
label_map = {lable:num for num,lable in enumerate(actions)}
for action in actions:
  action_file = os.listdir(os.path.join(DATA_PATH,action))
  for sequence in range(len(action_file)):#should change to the number of file
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH,action,str(sequence),f"{frame_num}.npy"))
      window.append(res)
    sequences.append(window)
    lables.append(label_map[action])
    
X = np.array(sequences)
y = to_categorical(lables).astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)
  

**Train the AI**

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflowjs as tfjs

In [8]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
actions.shape[0]

21

In [13]:
model = Sequential() #
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) #frame and all data shape
model.add(LSTM(128,return_sequences=True,activation='relu')) 
model.add(LSTM(64,return_sequences=False,activation='relu')) # return false because next layer is dense
model.add(Dense(64,activation='relu')) 
model.add(Dense(32,activation='relu')) 
model.add(Dense(actions.shape[0],activation='softmax')) # return output by how many action do we have => [0.7,0.1,0.2] / likely be 'hello'
#loss = categorical => multiple class 
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
#epochs increase relatively the number of data 
model.fit(X_train,y_train,epochs=100,callbacks=[tb_callback])

Epoch 1/100
19/19 [==============================] - 12s 357ms/step - loss: 3.2392 - categorical_accuracy: 0.0462
Epoch 2/100
19/19 [==============================] - 5s 278ms/step - loss: 2.9925 - categorical_accuracy: 0.0429
Epoch 3/100
19/19 [==============================] - 5s 250ms/step - loss: 2.7991 - categorical_accuracy: 0.1089
Epoch 4/100
19/19 [==============================] - 5s 241ms/step - loss: 2.6036 - categorical_accuracy: 0.1436
Epoch 5/100
19/19 [==============================] - 4s 237ms/step - loss: 2.5796 - categorical_accuracy: 0.1601
Epoch 6/100
19/19 [==============================] - 5s 241ms/step - loss: 2.5270 - categorical_accuracy: 0.1221
Epoch 7/100
19/19 [==============================] - 5s 263ms/step - loss: 2.4066 - categorical_accuracy: 0.1650
Epoch 8/100
19/19 [==============================] - 5s 269ms/step - loss: 2.2360 - categorical_accuracy: 0.2195
Epoch 9/100
19/19 [==============================] - 5s 254ms/step - loss: 2.1722 - categorical

In [15]:
#watch the log
%cd logs/train
%tensorboard --logdir=.

d:\Code\TF_SignLanguage\logs\train


UsageError: Line magic function `%tensorboard` not found.


In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_10 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_11 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_10 (Dense)             (None, 21)                693       
Total params: 597,269
Trainable params: 597,269
Non-trainable params: 0
________________________________________________

**save module**

In [20]:
model.save("action.h5")
tfjs.converters.save_keras_model(model, "tfjs")

In [ ]:
# reload model weight
model.load_weights('action.h5')

In [ ]:
#load model
from tensorflow import keras
model = keras.models.load_model('path/to/location')

**eval**

In [19]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()
multilabel_confusion_matrix(ytrue,yhat)
accuracy_score(ytrue,yhat)

0.84375

In [21]:
mp_holistic = mp.solutions.holistic # holistic module
mp_drawing = mp.solutions.drawing_utils # drawing utilities
def mediapipe_detection(image,model):
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # color conversion
  image.flags.writeable = False
  results = model.process(image) # process image
  image.flags.writeable = True
  image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # color conversion
  return image,results
def draw_landmarks(image,result):
  mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
  mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
def draw_styles_landmarked(image,result):
  mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                            mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                            mp_drawing.DrawingSpec(color=(80,256,10),thickness=1,circle_radius=1))
  mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
def extract_keypoint(result):
  pose = np.array([[res.x,res.y,res.z,res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
  face = np.array([[res.x,res.y,res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
  left_hand = np.array([[res.x,res.y,res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
  right_hand = np.array([[res.x,res.y,res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
  return np.concatenate([pose,face,left_hand,right_hand])

In [26]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoint(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()